In [1]:
import os
import yaml
import tqdm

import ase
from ase import io
from ase import data
from ase import db
from ase.visualize.plot import plot_atoms

from skimage import feature
import numpy as np

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

import ml_collections
from clu import checkpoint


from molnet import utils, train_state, train
from molnet.data import input_pipeline_online
from molnet.models import create_model
from configs import root_dirs
from analyses import make_predictions

from typing import Tuple

INDEX_TO_SYMBOL = {
    0: 'H',
    1: 'C',
    2: 'N',
    3: 'O',
    4: 'F'
}

2025-02-05 14:36:17.092657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
workdir = "/u/79/kurkil1/unix/work/molnet/runs/bf16-augs-rebias-adam-3e-4-z10-reverse-z/"
#workdir = "/u/79/kurkil1/unix/work/molnet/runs/bf16-augs-rebias-adam-3e-4-z20-interp/"
#workdir = "/Users/kurkil1/work/molnet/runs/attention-adam-3e-4/"

In [3]:
def load_from_workdir(
    workdir: str,
    return_attention: bool
):
    # Load the model config
    with open(os.path.join(workdir, "config.yaml"), "rt") as f:
        config = yaml.unsafe_load(f)
    config = ml_collections.ConfigDict(config)
    config.root_dir = root_dirs.get_root_dir("afms_rebias")
    config.model.return_attention_maps = return_attention

    print(config)

    # Create the model
    model = create_model(config.model)

    checkpoint_dir = os.path.join(workdir, "checkpoints")
    ckpt = checkpoint.Checkpoint(checkpoint_dir)

    apply_fn = model.apply
    tx = utils.create_optimizer(config)
    restored_state = ckpt.restore(state=None)['state']

    # Load the model state
    state = train_state.EvaluationState.create(
        apply_fn=apply_fn,
        params=restored_state['params'],
        batch_stats=restored_state['batch_stats'],
        tx=tx,
    )
    state = jax.tree_util.tree_map(jnp.asarray, state)

    return state, config

In [4]:
state, config = load_from_workdir(
    workdir=workdir,
    return_attention=False
)

batch_size: 12
cutout_probs:
- 0.5
- 0.3
- 0.1
- 0.05
- 0.05
dataset: afms_rebias
debug: false
eval_every_steps: 2000
gaussian_factor: 5.0
interpolate_input_z: null
learning_rate: 0.0003
learning_rate_schedule: constant
learning_rate_schedule_kwargs:
  decay_steps: 50000
  init_value: 0.0003
  peak_value: 0.0006
  warmup_steps: 2000
log_every_steps: 100
loss_fn: mse
max_atoms: 54
max_shift_per_slice: 0.02
model:
  attention_activation: sigmoid
  attention_channels:
  - 32
  - 32
  - 32
  - 32
  - 32
  conv_activation: relu
  decoder_channels:
  - 256
  - 128
  - 64
  - 32
  - 16
  decoder_kernel_size:
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  dtype: bfloat16
  encoder_channels:
  - 16
  - 32
  - 64
  - 128
  - 256
  encoder_kernel_size:
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  - - 3
    - 3
    - 3
  model_name: Attention-UNet
  output_activation: nul

2025-02-05 14:36:43.294256: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
rng = jax.random.PRNGKey(0)
datarng, rng = jax.random.split(rng)
with config.unlocked():
    #config.z_cutoff = 1.0
    #config.interpolate_z = None
    config.target_z_cutoff = 1.0
    #config.train_molecules = (0, 80000)
    #config.val_molecules = (80000, 100000)
    #config.max_shift_per_slice = 0.02

ds = input_pipeline_online.get_full_molecule_datasets(
    config
)['val']

batch = next(ds)

for k, v in batch.items():
    print(k, v.shape)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 18085879115725111131


images (12, 128, 128, 10, 1)
xyz (12, 54, 5)
sw (12, 2, 3)
atom_map (12, 128, 128, 10, 5)


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17927960504738319970
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7709568092925371842
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t

In [6]:
def grid_to_mol(
    grid: jnp.ndarray,
    peak_threshold: float = 0.5,
    z_cutoff: float = 1.0,
) -> ase.Atoms:
    grid = grid[..., ::-1, :]

    peaks = feature.peak_local_max(
        grid,
        min_distance=5,
        exclude_border=0,
        threshold_rel=peak_threshold
    )

    xyz_from_peaks = peaks[:, [1, 0, 2]] * (.125, .125, .1)
    elem_from_peaks = peaks[:, 3]

    mol = ase.Atoms(
        positions=xyz_from_peaks,
        symbols=[INDEX_TO_SYMBOL[i] for i in elem_from_peaks],
        cell=[16, 16, 0],
    )
    mol.positions[:, 2] -= mol.get_positions()[:, 2].max() - z_cutoff
    return mol

In [7]:
@jax.jit
def predict(
    state,
    batch,
):
    inputs, targets = batch['images'], batch['atom_map']
    preds = state.apply_fn(
        {'params': state.params, 'batch_stats': state.batch_stats},
        inputs,
        training=False,
    )
    return inputs, targets, preds, batch["xyz"]

In [8]:
batch = next(ds)

In [9]:
inputs, targets, preds, xyz = predict(state, batch)

2025-02-05 14:36:48.733069: W external/xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.07GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-02-05 14:36:54.652687: W external/xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.66GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-02-05 14:36:54.925324: W external/xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.12GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-02-05 14:36:55.672451: W external/xla/xla/tsl/framework/bfc_allocato

In [ ]:
for i in range(inputs.shape[0]):
    fig = plt.figure(figsize=(20, 2))
    for z in range(preds.shape[-2]):
        ax = plt.subplot(2, 10, z+1)
        ax.imshow(inputs[i, ..., z, 0], origin='lower')

    plt.show()

In [ ]:
labels = ["H", "C", "N", "O", "F"]
for i in range(inputs.shape[0]):

    fig = plt.figure(figsize=(10, 4))

    for s in range(preds.shape[-1]):
        ax = plt.subplot(2, 5, s + 1)
        ax.imshow(preds[i, ..., s].sum(-1), origin='lower', cmap='viridis')

        if s == 0:
            ax.set_title("Predictions, H")
        else:
            ax.set_title(f"{labels[s]}")

        ax = plt.subplot(2, 5, s + 6)
        ax.imshow(targets[i, ..., s].sum(-1), origin='lower', cmap='viridis')

        if s == 0:
            ax.set_title("Targets, H")
        else:
            ax.set_title(f"{labels[s]}")
    
    plt.tight_layout()


In [ ]:
for i in range(inputs.shape[0]):
    fig = plt.figure(figsize=(20, 2))
    for z in range(preds.shape[-2]):
        ax = plt.subplot(2, 10, z+1)
        ax.imshow(preds[i, ..., z, :].sum(-1).T, origin='lower', vmin=0, vmax=5)

        ax = plt.subplot(2, 10, z+11)
        ax.imshow(targets[i, ..., z, :].sum(-1).T, origin='lower', vmin=0, vmax=5)

    plt.show()